# 言語理解（Language Understanding）

今後は、自然言語による音声や入力されたコマンドを理解するために、コンピュータがAIを利用できるようになることが期待されています。例えば、「ライトをつけて」「扇風機をつけて」などの音声コマンドで家庭内の機器を操作できるホームオートメーションシステムを導入し、AI搭載機器がそのコマンドを理解して適切な行動を取ることができるようにしたいと考えています。

![A robot listening](./images/language_understanding.jpg)

## オーサリングおよび予測リソースを作成する

MicrosoftのCognitive Serviceには、Language Understandingサービスが含まれています。これにより、**発話（utterances）** をベースとして **エンティティ（entities）** と、それに作用する **意図（intents）** を定義することができます。

Language Understandingサービスを利用するには、2種類のリソースが必要です。

- **作成(Authoring)** リソース: 言語モデルの定義、トレーニング、およびテストに使用されます。これは、Azureサブスクリプションの**Language Understanding - Authoring**リソースである必要があります。
- **予測**リソース: モデルを公開し、それを使用するクライアントアプリケーションからの要求を処理するために使用されます。これは、Azureサブスクリプション内の**Language Understanding**または**Cognitive Services**リソースのいずれかである必要があります。

Language Understandingアプリを公開するには、**Language Understanding**リソースまたは**Cognitive Services**リソースのどちらかを利用できますが、アプリの**オーサリング** には、それらとは別に **Language Understanding** リソースを作成する必要があります。

> **重要**： 作成リソースは、4つの**リージョン**（オーストラリア東部、西ヨーロッパ、米国西部、スイス北部）のいずれかで作成する必要があります。ヨーロッパまたはオーストラリアの作成リソースで作成されたモデルは、それぞれヨーロッパまたはオーストラリアの予測リソースにのみデプロイできます。米国の作成リソースで作成されたモデルは、ヨーロッパとオーストラリア以外のAzureのロケーションにある予測リソースにデプロイできます。作成と予測の場所のマッチングについては、[作成と公開のリージョン](https://docs.microsoft.com/azure/cognitive-services/luis/luis-reference-regions)を参照してください。

1. 新しいブラウザタブで、Azureポータル（https://portal.azure.com）を開き、Microsoftアカウントでサインインします。
2. **+ リソースの作成** をクリックして、**Language Understanding**を検索します。
3. サービスがリストから **Language Understanding** をクリックします。
4. **Language Understanding** の画面が表示されたら、**作成** をクリックします。
5. 次の画面では、以下の設定を行い **確認と作成**をクリックします。
   - **Create option**: Both
   
   - **Project Details**
     - **サブスクリプション**: *ご自身のサブスクリプション*
     - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
   - **Instance Details**
     - **Name**: *ご自身のサービスのユニークな名前*
   - **Authoring Resource**
     - **リージョン**: *好みのリージョンを選択*
     - **Authoring pricing tier**: F0
   - **Prediction Resource**
     - **Prediction location**: *Authoring Resource で設定したリージョンと同じリージョンを選択*
     - **Prediction pricing tier**: F0
6. 設定の検証が成功したら**作成**をクリックします。
7. しばらく待ってリソースが作成されると、作成用と予測用の2つのLanguage Understandingリソースがプロビジョニングされます。これらのリソースは、リソースを作成した**リソースグループに移動**することで確認できます。

### Language Understandingアプリを作成する

Language Understandingで自然言語理解を実装するには、アプリを作成し、エンティティ、意図、発話を追加して、アプリに理解させたいコマンドを定義します。

1. 新しいブラウザタブを開き、作成リソースを作成した作成リージョンのLanguage Understandingポータルを開きます。
    - 米国: [https://www.luis.ai](https://www.luis.ai)
    - ヨーロッパ: [https://eu.luis.ai](https://eu.luis.ai)
    - オーストラリア: [https://au.luis.ai](https://au.luis.ai)

2. お客様のAzureサブスクリプションに関連付けられたMicrosoftアカウントを使用してサインインします。初めてLanguage Understandingポータルにサインインした場合は、アカウントの詳細にアクセスする権限をアプリに付与する必要がある場合があります。次に、Azureサブスクリプションで作成したばかりの既存のLanguage Understandingオーサリングリソースを選択して、**Welcome**ステップを完了します。

3. **Converation Apps** のページを開き、ご利用のサブスクリプションとLanguage Understanding作成リソースを選択します。次に、以下の設定で会話用の新しいアプリを作成します。
   - **Name**: Home Automation
   - **Culture**: English (*このオプションが利用できない場合は空白にしてください。*)
   - **Description**: Simple home automation
   - **Prediction resource**: *前のステップで作成した Language Understanding予測リソース*

4. 効果的なLanguage Understandingアプリを作成するためのヒントが書かれたパネルが表示された場合は、それを閉じます。

### Create an Entity

An *entity* is a thing that your language model can identify and do something with. In this case, your Language Understanding app will be used to control various *devices* in the office, such as lights or fans; so you'll create a *device* entity that includes a list of the types of device that you want the app to work with. For each device type, you'll create a sublist that identifies the name of the device (for example *light*) and any synonyms that might be used to refer to this type of device (for example *lamp*).

1. In the Language Understanding page for your app, in the pane on the left, click **Entities**. Then click **Create**, and create a new entity named **device**, select the **List** type, and click **Create**.
2. In the **List items** page, under **Normalized Values**, type **light**, then press ENTER.
3. After the **light** value has been added, under **Synonyms**, type **lamp** and press ENTER.
4. Add a second list item named **fan** with the synonym **AC**.

> **Note**: For this lab, use the exact lowercase or uppercase text as instructed _(example: light **not** Light)_ and do not add extra spaces. 

### Create Intents

An *intent* is an action you want to perform on one or more entities - for example, you might want to switch a light on, or turn a fan off. In this case, you'll define two intents: one to switch a device on, and another to switch a device off. For each intent, you'll specify sample *utterances* that indicate the kind of language used to indicate the intent.

> **Note**: For this lab, please use the exact lowercase or uppercase text as instructed _(example: "turn the light on" **not** "Turn the light on .")_ and do not add extra spaces. 

1. In the pane on the left, click **Intents**. Then click **Create**, and add an intent with the name **switch_on** and click **Done**.
2. Under the **Examples** heading and the **Example user input** subheading, type the utterance ***turn the light on*** and press **Enter** to submit this utterance to the list.
3. In the *turn the light on* utterance, click the word "light", and assign it to the **device** entity's **light** value.

![How to assign the word "light" to the entity value.](./images/assign_entity.jpg)

4. Add a second utterance to the **switch_on** intent, with the phrase ***turn the fan on***. Then assign the word "fan" to the **device** entity's **fan** value.
5. In the pane on the left, click **Intents** and click **Create**, to add a second intent with the name **switch_off**.
6. In the **Utterances** page for the **switch_off** intent, add the utterance ***turn the light off*** and assign the word "light" to the **device** entity's **light** value.
7. Add a second utterance to the **switch_off** intent, with the phrase ***turn the fan off***. Then connect the word "fan" to the **device** entity's **fan** value.

### Train and Test the Language Model

Now you're ready to use the data you've provided in the form of entities, intents, and utterances to train the language model for your app.

1. At the top of the Language Understanding page for your app, click **Train** to train the language model
2. When the model is trained, click **Test**, and use the Test pane to view the predicted intent for the following phrases:
    * *switch the light on*
    * *turn off the fan*
    * *turn the lamp off*
    * *switch on the AC*
3. Close the Test pane.
    
### Publish the Model and Configure Endpoints

To use your trained model in a client application, you must publish it as an endpoint to which the client applications can send new utterances; from which intents and entitites will be predicted.

1. At the top of the Language Understanding page for your app, click **Publish**. Then select **Production slot** and click **Done**.

2. After the model has been published, at the top of the Language Understanding page for your app, click **Manage**. Then on the **Settings** tab, note the **App ID** for your app. Copy this and paste it in the code below to replace **YOUR_LU_APP_ID**.

3. On the **Azure Resources** tab, note the **Primary key** and **Endpoint URL** for your prediction resource. Copy these and paste them into the code below, replacing **YOUR_LU_KEY** and **YOUR_LU_ENDPOINT**.

4. Run the cell below by clicking its **Run cell** (&#9655;) button (to the left of the cell), and when prompted, enter the text *turn the light on*. The text is interpreted by your Language Understanding model and an appropriate image is displayed.

### **(!) Important**: 
Look for the prompt at the top of your window. You will need to type *turn the light on* and press **enter**. 


In [ ]:
from python_code import luis
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

try:
    # Set up API configuration
    luis_app_id = 'YOUR_LU_APP_ID'
    luis_key = 'YOUR_LU_KEY'
    luis_endpoint = 'YOUR_LU_ENDPOINT'

    # prompt for a command
    command = input('Please enter a command: \n')

    # get the predicted intent and entity (code in python_code.home_auto.py)
    action = luis.get_intent(luis_app_id, luis_key, luis_endpoint, command)

    # display an appropriate image
    img_name = action + '.jpg'
    img = Image.open(os.path.join("data", "luis" ,img_name))
    plt.axis('off')
    plt. imshow(img)
except Exception as ex:
    print(ex)

### (!) Check In 
Did you run the cell above, then type the phrase *turn the light on* when prompted? The prompt will appear at the top of your window.  

Re-run the cell above, trying the following phrases:

* *turn on the light*
* *put the lamp off*
* *switch the fan on*
* *switch the light on*
* *switch off the light*
* *turn off the fan*
* *switch the AC on*

If you ran the cell above and it displayed question mark image, you may have used slightly different text or spacing than instructed when you created an entity, intent, or utterance.

> **Note**: If you're curious about the code used to retrieve the intents and entitites from your Language Understanding app, look at the **luis.py** file in the **python_code** folder.

## Add Voice Control

So far, we've seen how analyze text; but increasingly AI systems enable humans to communicate with software services through speech recognition. To support this, the **Speech** cognitive service provides a simple way to transcribe spoken language into text.

### Create a Cognitive Services Resource

If you don't already have one, use the following steps to create a **Cognitive Services** resource in your Azure subscription:

> **Note**: If you already have a Cognitive Services resource, just open its **Quick start** page in the Azure portal and copy its key and location to the cell below. Otherwise, follow the steps below to create one.

1. In another browser tab, open the Azure portal at [https://portal.azure.com](https://portal.azure.com), signing in with your Microsoft account.
2. Click the **&#65291;Create a resource** button, search for *Cognitive Services*, and create a **Cognitive Services** resource with the following settings:
    - **Subscription**: *Your Azure subscription*.
    - **Resource group**: *Select or create a resource group with a unique name*.
    - **Region**: *Choose any available region*:
    - **Name**: *Enter a unique name*.
    - **Pricing tier**: S0
    - **By checking this box, I certify that use of this service is not by or for a police department in the United States**: Selected.
    - **I confirm I have read and understood the notices**: Selected.
3. Wait for deployment to complete. Then go to your cognitive services resource, and on the **Quick start** page, note the keys and location. You will need these to connect to your cognitive services resource from client applications.

### Get the Key and Location for your Cognitive Services Resource

To use your cognitive services resource, client applications need its authentication key and location:

1. In the Azure portal, on the **Keys and Endpoint** page for your cognitive service resource, copy the **Key1** for your resource and paste it in the code below, replacing **YOUR_COG_KEY**.
2. Copy the **Location** for your resource and paste it in the code below, replacing **YOUR_COG_LOCATION**.
>**Note**: Stay on the **Keys and Endpoint** page and copy the **Location** from this page (example: _westus_). Please _do not_ add spaces between words for the Location field. 
3. Run the code in the cell below. 

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

Now run the cell below to transcribe speech from an audio file, and use it as a command for your Language Understanding app.

In [ ]:
import os
from python_code import luis
from azure.cognitiveservices.speech import SpeechConfig, SpeechRecognizer, AudioConfig
from playsound import playsound
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

try:   

    # Get spoken command from audio file
    file_name = 'light-on.wav'
    audio_file = os.path.join('data', 'luis', file_name)

    # Configure speech recognizer
    speech_config = SpeechConfig(cog_key, cog_location)
    audio_config = AudioConfig(filename=audio_file) # Use file instead of default (microphone)
    speech_recognizer = SpeechRecognizer(speech_config, audio_config)

    # Use a one-time, synchronous call to transcribe the speech
    speech = speech_recognizer.recognize_once()

    # Get the predicted intent and entity (code in python_code.home_auto.py)
    action = luis.get_intent(luis_app_id, luis_key, luis_endpoint, speech.text)

    # Get the appropriate image
    img_name = action + '.jpg'

    # Display image 
    img = Image.open(os.path.join("data", "luis" ,img_name))
    plt.axis('off')
    plt. imshow(img)
    playsound(audio_file)

except Exception as ex:
    print(ex)

Try modifying the cell above to use the **light-off.wav** audio file.

## Learn More

Learn more about Language Understanding in the [service documentation](https://docs.microsoft.com/azure/cognitive-services/luis/)